In [150]:
import pandas as pd
import numpy as np

from utils import TABLES

In [164]:
df = pd.read_excel("raw_data_source.xlsx", sheet_name = TABLES[2])

In [195]:
def format_value(value):
#     print(value)
    if value is None or value == np.nan:
        return "NULL"
    
    elif isinstance(value, str):
        return f"'{value}'"
    
    else:
        return str(value)

def dataframe_to_sql_insert(dataframe, table_name):
    insert_statements = []

    for index, row in dataframe.iterrows():
        if index != 0:
            values = ', '.join([format_value(value) for value in row])
            insert_statement = f"INSERT INTO {table_name} VALUES ({values});"
            insert_statement = insert_statement.replace("nan", "NULL")
            insert_statements.append(insert_statement)
    
    return '\n'.join(insert_statements)

In [196]:
print(dataframe_to_sql_insert(df, "Employees"))

INSERT INTO Employees VALUES (2, 'Fuller', 'Andrew', 'Vice President, Sales', 'Dr.', 1952-02-19 00:00:00, 1992-08-14 00:00:00, '908 W. Capital Way', 'Tacoma', 'WA', '98401', 'USA', '(206) 555-9482', 3457, 'Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981.  He is fluent in French and Italian and reads German.  He joined the company as a sales representative, was promoted to sales manager in January 1992 and to vice president of sales in March 1993.  Andrew is a member of the Sales Management Roundtable, the Seattle Chamber of Commerce, and the Pacific Rim Importers Association.', NULL, 'http://accweb/emmployees/fuller.bmp');
INSERT INTO Employees VALUES (3, 'Leverling', 'Janet', 'Sales Representative', 'Ms.', 1963-08-30 00:00:00, 1992-04-01 00:00:00, '722 Moss Bay Blvd.', 'Kirkland', 'WA', '98033', 'USA', '(206) 555-3412', 3355, 'Janet has a BS degree in chemistry from Boston College (1984).  She has also completed a

In [87]:
import pyodbc 
import json

In [128]:
def connect():
    
    with open('pydb_config.json', 'r') as f:
        config = json.load(f)
    
    DRVIER = f"DRIVER=" + config.get("driver")
    SERVER = f"SERVER=" + config.get("server")
    DATABASE = f"DATABASE=" + config.get("database")
    UID = f"UID=" + config.get("username")
    PWD = f"PWD=" + config.get("password")
    ENCRYPT = "ENCRYPT=yes"
    TRUST = "TrustServerCertificate=yes"

    connection_params = [DRVIER, SERVER, DATABASE, ENCRYPT, TRUST, UID, PWD]
    connection_string = ";".join(connection_params)
    
    connection = pyodbc.connect(connection_string, autocommit = True)
    
    return connection

In [129]:
def execute(filename, cursor):
    with open(filename, "r") as file:
        content = file.read()
        cursor.execute(content)

In [130]:
connection = connect()
cursor = connection.cursor()

In [146]:
execute("database_creation.sql", cursor)

In [147]:


for table in TABLES:
    execute(f"create_table_{table}.sql", cursor)
    print("Table created:", table)


Table created: Categories
Table created: Customers
Table created: Employees
Table created: OrderDetails
Table created: Orders
Table created: Products
Table created: Region
Table created: Shippers
Table created: Suppliers
Table created: Territories


In [148]:
primary_keys_script = "add_primary_key_constraints.sql"
foreign_keys_script = "add_foreign_key_constraints.sql"

execute(primary_keys_script, cursor)
print("Primary keys added")

execute(foreign_keys_script, cursor)
print("Foreign keys added")

Primary keys added
Foreign keys added


In [149]:
# execute("drop_foreign_key_constraints.sql", cursor)